In [1]:
import tensorflow as tf
import numpy as np

/home/hojoon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/hojoon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Preprocess Data

In [2]:
def word2idx(word, max_length):
    idxs = []
    for c in word:
        idx = ord(c) - 96
        idxs.append(idx)
        
    # Pad until the max_length
    remain = max_length - len(idxs)
    for _ in range(remain):
        idxs.append(0)
    
    return idxs


def one_hot(wordIdx):
    
    return np.eye(27)[wordIdx]    

In [3]:
def preprocess(words):
    X  = []
    Y  = []
    
    max_length = len(max(words, key = len))
    
    for word in words:
        X.append(one_hot(word2idx(word[:-1], max_length)))
        Y.append(ord(word[-1])-96)
    
    return np.array(X), np.array(Y,dtype=np.int32)

In [4]:
words = ['word','wood','deep','dive','cold','cool','load','love','kiss','king','queen'
         'hello','hi','brother','sister','mom','dad','mother','father','machine','learning']

input_batch, target_batch = preprocess(words)
print(input_batch.shape)
print(target_batch.shape)

(20, 10, 27)
(20,)


### Configuration

In [5]:
learning_rate = 1e-2
total_epoch = 30
batch_size  = 256


input_size  = class_size = 27
hidden_size = 128
step_size   = len(max(words, key = len))

### Build Graph

In [6]:
X = tf.placeholder(tf.float32, [None, step_size, input_size])
Y = tf.placeholder(tf.int32, [None])

In [7]:
W = tf.Variable(tf.random_normal([hidden_size, class_size]))
b = tf.Variable(tf.random_normal([class_size]))

In [8]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob= 0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)

In [9]:
cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(cell = cell, inputs = X, dtype = tf.float32)

print(outputs.shape)

(?, 10, 128)


In [10]:
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

score = tf.matmul(outputs, W) + b

In [11]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = score, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op  = optimizer.minimize(loss)

In [13]:
# Prediction
prediction = tf.cast(tf.argmax(score, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.to_float(prediction_check))

### Train Graph

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(total_epoch):            
        _, total_loss = sess.run([train_op, loss], feed_dict = {X:input_batch, Y:target_batch})
        
        print('Epoch:','%04d' %(epoch+1),
              'Avg Loss: %.3f' %(total_loss / len(input_batch)))

        
    #Prediction
    print('\n Accuracy: ',sess.run(accuracy, feed_dict = {X:input_batch, Y:target_batch}))

Epoch: 0001 Avg Loss: 0.194
Epoch: 0002 Avg Loss: 0.120
Epoch: 0003 Avg Loss: 0.131
Epoch: 0004 Avg Loss: 0.095
Epoch: 0005 Avg Loss: 0.053
Epoch: 0006 Avg Loss: 0.055
Epoch: 0007 Avg Loss: 0.039
Epoch: 0008 Avg Loss: 0.022
Epoch: 0009 Avg Loss: 0.025
Epoch: 0010 Avg Loss: 0.012
Epoch: 0011 Avg Loss: 0.024
Epoch: 0012 Avg Loss: 0.008
Epoch: 0013 Avg Loss: 0.056
Epoch: 0014 Avg Loss: 0.015
Epoch: 0015 Avg Loss: 0.020
Epoch: 0016 Avg Loss: 0.047
Epoch: 0017 Avg Loss: 0.009
Epoch: 0018 Avg Loss: 0.001
Epoch: 0019 Avg Loss: 0.008
Epoch: 0020 Avg Loss: 0.006
Epoch: 0021 Avg Loss: 0.010
Epoch: 0022 Avg Loss: 0.004
Epoch: 0023 Avg Loss: 0.012
Epoch: 0024 Avg Loss: 0.008
Epoch: 0025 Avg Loss: 0.003
Epoch: 0026 Avg Loss: 0.017
Epoch: 0027 Avg Loss: 0.001
Epoch: 0028 Avg Loss: 0.000
Epoch: 0029 Avg Loss: 0.008
Epoch: 0030 Avg Loss: 0.018

 Accuracy:  1.0
